# **Installing pinecone**

---



In [1]:
!pip install --upgrade pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 5.5 MB/s eta 0:00:00


# **Installing pre-trained Transformers for better and easy implementation**

---



In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 7.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

# **Importing necessary libraries**

---


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
from pinecone import Pinecone, ServerlessSpec

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# **Defining Pinecone**

---



In [4]:
pc = Pinecone(api_key="1fa98373-be29-457c-8dbf-777414f715d1")

index_name = "my-index"
if index_name not in pc.list_indexes().names():
  # try:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="euclidean",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
  # except TypeError:
  #    pass  # Ignore the TypeError for now

index = pc.Index(index_name)

# **Importing Document**

---
Not using file handling to import document because it is colab environment


In [5]:
documents=['''
Introduction
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving.
''',
'''Types of AI

Narrow AI (Weak AI)
Designed and trained for a particular task. Examples include voice assistants like Siri and Alexa, recommendation systems on streaming services, and self-driving cars.

General AI (Strong AI)
Possesses generalized human cognitive abilities. When presented with an unfamiliar task, a strong AI system can find a solution without human intervention.

Superintelligent AI
Surpasses human intelligence and capability in all fields, including creativity, general wisdom, and social skills.
''',
'''History of AI

1950s: Alan Turing proposed the "Turing Test" to determine if a machine can exhibit intelligent behavior indistinguishable from a human.
1956: The term "Artificial Intelligence" was coined by John McCarthy during the Dartmouth Conference.
1960s-1970s: Development of early AI programs like ELIZA (a simple chatbot) and the first AI-based chess game.
1980s: Introduction of machine learning, where systems could learn and improve from experience.
1997: IBM's Deep Blue defeated world chess champion Garry Kasparov.
2010s: Advancements in deep learning and neural networks led to significant improvements in AI capabilities.
''',
'''Key Concepts and Technologies in AI

Machine Learning (ML)
A subset of AI that involves the use of algorithms and statistical models to perform specific tasks without explicit instructions, relying on patterns and inference instead.

Deep Learning
A subset of ML that uses neural networks with many layers (hence "deep") to analyze various factors of data.

Neural Networks
Inspired by the human brain, these networks consist of neurons (nodes) and are designed to recognize patterns.

Natural Language Processing (NLP)
Enables machines to understand, interpret, and respond to human language. Examples include chatbots, sentiment analysis, and language translation.

Computer Vision
Allows machines to interpret and make decisions based on visual input. Applications include facial recognition, autonomous driving, and medical image analysis.

Robotics
Involves designing and creating robots that can perform tasks autonomously or semi-autonomously. Examples include industrial robots, drones, and robotic vacuum cleaners.

Reinforcement Learning
A type of ML where an agent learns to make decisions by performing actions and receiving rewards or penalties.
''',
'''Applications of AI

Healthcare
AI is used in medical imaging, drug discovery, personalized medicine, and virtual health assistants.

Finance
AI algorithms are employed in fraud detection, algorithmic trading, personalized banking, and risk management.

Transportation
Autonomous vehicles, traffic management systems, and logistics optimization.

Retail
Recommendation systems, customer service chatbots, inventory management, and personalized marketing.

Entertainment
Content recommendation (Netflix, Spotify), game development, and virtual influencers.

Manufacturing
Predictive maintenance, quality control, and robotics automation.

Education
Adaptive learning platforms, automated grading, and AI tutors.

Agriculture
Precision farming, crop monitoring, and automated harvesting.
''',
'''
Challenges in AI

Ethical Considerations
Bias and fairness, privacy concerns, and the potential for misuse.

Job Displacement
Automation of tasks leading to potential unemployment.

Security
AI systems can be vulnerable to hacking and other security threats.

Transparency and Accountability
Understanding and explaining how AI models make decisions.

Regulation
Establishing guidelines and laws to ensure the safe and ethical use of AI.
''',
'''
Future of AI

Advancements in AI Research
Continued progress in areas like general AI, human-AI collaboration, and AI safety.

AI in Daily Life
Increased integration of AI in everyday devices and services.

AI and Society
Ongoing discussions about the societal impact of AI, including its role in decision-making and governance.
''',
'''
Conclusion
AI is transforming various sectors and reshaping the world as we know it. While it offers tremendous benefits, it also presents significant challenges that must be addressed to ensure its responsible and ethical development. Understanding AI's capabilities, applications, and potential impacts is crucial for navigating its future.''']

# **Defining SentenceTransformer Model**

---



In [6]:
Model=SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Embedding of Document**

---



In [7]:
embeddings=Model.encode(documents)

In [8]:
for doc_id, embedding in enumerate(embeddings):
    index.upsert([(str(doc_id), embedding)])

# **Embedding Query string**

---



In [9]:
query='what is mean by AI' # input('enter you Query here: ')
query_embedding=Model.encode([query])[0]

query_embedding = query_embedding.tolist()    # Converting numpy array to List

results=index.query(vector=query_embedding, top_k=5)         # Retrieving top 5 most similar documents

retrieved_documents=[documents[int(res['id'])] for res in results['matches']]

In [10]:
retrieved_documents

['\nIntroduction\nArtificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving.\n',
 'Applications of AI\n\nHealthcare\nAI is used in medical imaging, drug discovery, personalized medicine, and virtual health assistants.\n\nFinance\nAI algorithms are employed in fraud detection, algorithmic trading, personalized banking, and risk management.\n\nTransportation\nAutonomous vehicles, traffic management systems, and logistics optimization.\n\nRetail\nRecommendation systems, customer service chatbots, inventory management, and personalized marketing.\n\nEntertainment\nContent recommendation (Netflix, Spotify), game development, and virtual influencers.\n\nManufacturing\nPredictive maintenance, quality control, and robotics automation.\n\nEducation\nAdaptive learning plat

# **Installing (GPT2LMHeadModel & GPT2Tokenizer) for pretrained models**

---



In [11]:
!pip install transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# **Generating texts from GPT model**

---

Ensuring to split Unecessary informations. like Retrived document data's and else.

In [12]:
model_name='gpt2'
tokenizer=GPT2Tokenizer.from_pretrained(model_name)
model=GPT2LMHeadModel.from_pretrained(model_name)

context=" ".join(retrieved_documents)
prompt=f"Based on the following information: {context}\n\n split. Question: {query}\nAnswer (in a concise and clear manner):"

inputs=tokenizer.encode(prompt,return_tensors='pt',max_length=512, truncation=True)
outputs=model.generate(
    inputs,max_length=512,num_return_sequences=1,no_repeat_ngram_size=2,do_sample=True,top_p=0.95,top_k=50
)

response=tokenizer.decode(outputs[0],skip_special_tokens=True)

if "split." in response:
    answer=response.split("split.")[1].strip()
else:
    answer=response.strip()

# Post-process to remove any extraneous text or sentences
# answer=answer.split('.')[0] + '.'
print(answer)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: what is AI
Answer (in a concise and clear manner): What is its purpose? The title of this paper describes its purposes and its principles, with the emphasis in particular on how it might.
